# Spectrum

In this notebook we will analyse the energy distribution of the events. Along the way we will investigate if the gamma-ray emission is statistically significant, and we will take a look at the development of the significance over time.

We will need the significance map and exclusion mask from the previous step. Our final product will be the energy spectrum of the source in the form of spectral data points, a spectral model that describes best the observed data and a list of data sets for time-dependent analysis at a later stage.

## Imports

Let's start with importing ther modules and classes that we have seen in the last session.

In [ ]:
import os

import matplotlib.pyplot as plt

import numpy as np

from numpy import sqrt

import astropy.units as u

from astropy.coordinates import (
    SkyCoord, 
    Angle,
)

from gammapy.utils.check import check_tutorials_setup

from gammapy.data import (
    DataStore,
)

from gammapy.stats import WStatCountsStatistic

from gammapy.maps import (
    Map,
    MapAxis, 
    WcsGeom,
)

In [ ]:
from regions import CircleSkyRegion

## Check the Data

In [ ]:
if not os.path.exists('gammapy-data'):
    check_tutorials_setup()
else:
    print('Great your setup is correct!')

In [ ]:
os.environ['GAMMAPY_DATA'] = 'gammapy-data/1.2'

In [ ]:
data_store = DataStore.from_dir('$GAMMAPY_DATA/hess-dl3-dr1')

In [ ]:
data_store.info()

## Run Selection

In [ ]:
source_pos = SkyCoord(83.633*u.deg, 22.014*u.deg, frame='icrs')

In [ ]:
selectradius = 2.5*u.deg

In [ ]:
conesearch = data_store.obs_table.select_sky_circle(source_pos, selectradius)

In [ ]:
runlist = conesearch['OBS_ID'].value

In [ ]:
observations = data_store.get_observations(runlist)

In [ ]:
observations.ids

In [ ]:
obs = observations[0]

In [ ]:
obs

## Motivation

Remember that we plotted the energy distribution of all the events in one observation run:

In [ ]:
obs.events.plot_energy()

#plt.savefig('Counts_vs_Energy.svg')

These event counts were recorded within a certain observation time. We need to divide the counts by the time to get a rate (counts per second).

In [ ]:
obs.observation_live_time_duration

But we want to measure a flux. So we still need to divide by the effective area of the instrument. The effective area depends on the energy, but also on the conditions of the observations (like the distance of the source from the camera centre).

In [ ]:
obs.aeff.plot_energy_dependence()

plt.xlim(0.4)

#plt.savefig('Aeff.svg')

And we will need to consider the energy resolution and dispersion.

In [ ]:
obs.edisp.peek()

In [ ]:
obs.edisp.to_edisp_kernel(0.7*u.deg).plot_matrix()

#plt.savefig('EDisp.svg')

In [ ]:
obs.edisp.plot_migration(offset = 0*u.deg, energy_true = [1,10]*u.TeV)

plt.legend(loc='upper right')

#plt.savefig('Eresolution.svg')

All this can be handled with a set of makers.

## Makers for the creation of a Spectrum Dataset

### SpectrumDatasetMaker
We will now use gammapy code to do the binning of our event lists. This is very similar to the generation of the MapDataset.

In [ ]:
from gammapy.datasets import SpectrumDataset

from gammapy.makers import SpectrumDatasetMaker

We want to use only the events from a small region around the source, the on region. We start with a region at the source position and with a radius of 0.1 degrees. We can change that later if needed.

In [ ]:
on_region_radius = Angle(0.14*u.deg)

on_region = CircleSkyRegion(center=source_pos, radius=on_region_radius)

We can check on the sky map if the region really encompasses all of the emission. If not, we we need to increase the size. We will use the significance map created in the last step and zoom in.

In [ ]:
significance_map = Map.read('SigMap.fits.gz')

In [ ]:
zoomed_significance_map = significance_map.cutout(source_pos, 1*u.deg)

In [ ]:
ax = zoomed_significance_map.plot(add_cbar = True, cmap = 'plasma')

on_region.to_pixel(ax.wcs).plot(color = 'white', ax = ax)

In [ ]:
# minimum and maximum energy for the analysis
Emin = 0.1*u.TeV
Emax = 50*u.TeV

In [ ]:
spectrum_nEbins = 20

spectrum_energy_axis = MapAxis.from_energy_bounds(Emin, Emax,
                                                  nbin=spectrum_nEbins,
                                                  name="energy")

In [ ]:
#We will also need an axis for true energy:

energy_axis_true = MapAxis.from_energy_bounds(0.08*u.TeV, 80*u.TeV,
                                              nbin=8,
                                              per_decade=True,
                                              name='energy_true')

In [ ]:
from gammapy.maps import RegionGeom

In [ ]:
spectrum_geom = RegionGeom.create(region=on_region, 
                                  axes=[spectrum_energy_axis]
                                 )

In [ ]:
spectrum_geom

In [ ]:
spectrum_empty =  SpectrumDataset.create(geom=spectrum_geom, 
                                         energy_axis_true=energy_axis_true
                                        )

In [ ]:
from gammapy.makers import SpectrumDatasetMaker

In [ ]:
spectrum_maker = SpectrumDatasetMaker()

In [ ]:
spectrum_dataset = spectrum_maker.run(spectrum_empty, obs)

In [ ]:
spectrum_dataset.counts.plot()

In [ ]:
#spectrum_dataset.counts_off.plot()

In [ ]:
spectrum_dataset.mask.plot()

### SafeMaskMaker
As before, we need to select the good energy and offset range. We will use the same SafeMaskMaker as before.

In [ ]:
from gammapy.makers import SafeMaskMaker

In [ ]:
# maximumum offset
offset_max = 2.5*u.deg

aeff_max = 10

In [ ]:
safe_mask_maker = SafeMaskMaker(methods=['offset-max', 'aeff-max'], 
                                offset_max = offset_max,
                                aeff_percent = aeff_max
                               )

In [ ]:
spectrum_dataset = safe_mask_maker.run(spectrum_dataset, obs)

In [ ]:
spectrum_dataset.mask.plot()
plt.yscale('linear')

In [ ]:
spectrum_dataset.energy_range_total

In this run we will use only events with energies of more than 645 GeV.

### Reflected Background
We will use the reflected background to estimate the off source counts. We will need an exclusion mask, we will use the same as before.

In [ ]:
from gammapy.makers import ReflectedRegionsBackgroundMaker

In [ ]:
exclusion_mask = Map.read('ExclusionMask.fits.gz')

In [ ]:
# The mask did not properly save the data type (bool). But we can set it manually. If the mask is created in the same notebook then this step is not necessary.

exclusion_mask.data = exclusion_mask.data.astype(bool)

In [ ]:
exclusion_mask.plot()

In [ ]:
reflected_bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)

In [ ]:
spectrum_dataset = reflected_bkg_maker.run(spectrum_dataset, obs)

In [ ]:
spectrum_dataset.counts_off.plot()

In [ ]:
spectrum_dataset.alpha.plot()

We can check on the sky map the location of our off-source regions.

In [ ]:
from gammapy.visualization import plot_spectrum_datasets_off_regions

In [ ]:
ax = significance_map.plot()

on_region.to_pixel(ax.wcs).plot(ax = ax, color = 'white')

plot_spectrum_datasets_off_regions(ax=ax, datasets=[spectrum_dataset])

### Combination of all runs
As for the skyp maps, we will us the ```DatasetsMaker``` to loop over all observations.

In [ ]:
from gammapy.makers import DatasetsMaker

In [ ]:
chainmaker_spectrum = DatasetsMaker(makers = [spectrum_maker,
                                              safe_mask_maker,
                                              reflected_bkg_maker],
                               stack_datasets = False,
                               n_jobs = 4, parallel_backend = 'multiprocessing'
                               )

In [ ]:
spectrum_datasets = chainmaker_spectrum.run(
    dataset = spectrum_empty.copy(name = 'stacked'), ## this will not be used
    observations = observations)

Stacking the spectrum data set is much easier:

In [ ]:
spectrum_stacked = spectrum_datasets.stack_reduce()

In [ ]:
ax = significance_map.plot(cmap = 'plasma', add_cbar = True)

on_region.to_pixel(ax.wcs).plot(ax = ax, color = 'white')

plot_spectrum_datasets_off_regions(ax=ax, datasets=spectrum_datasets)

#plt.savefig('ReflectedRegions.svg')

### Excess and Significance
We can now use the SpectrumDatasets to study the excess and the significance of the source.

In [ ]:
info_table = spectrum_datasets.info_table()

In [ ]:
info_table.colnames

In [ ]:
info_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
#info_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts'].write('info_table.csv')

In [ ]:
sum_table = spectrum_datasets.info_table(cumulative=True)

In [ ]:
sum_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
#sum_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts'].write('sum_table.csv')

The last line gives us the on and off counts, as well as excess and significance.

In [ ]:
last_line = sum_table[-1]

In [ ]:
last_line['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
last_line['counts'] - last_line['alpha'] * last_line['counts_off']

In [ ]:
stat = WStatCountsStatistic(n_on=last_line['counts'], 
                            n_off=last_line['counts_off'], 
                            alpha=last_line['alpha']
                           )

print('excess: {} \nsignificance: {}'.format(stat.n_sig,stat.sqrt_ts))

The excess should increase roughly linearly with time:

In [ ]:
plt.errorbar(sum_table['livetime'].to(u.h),
             sum_table['excess'],
             np.sqrt(sum_table['excess']),
             marker='o',
             ls='none'
            )

plt.xlabel('Livetime [h]')
plt.ylabel('Excess')

#plt.savefig('ExcessVsT.svg')

And the significance with the sqrt of time:

In [ ]:
plt.errorbar(sum_table['livetime'].to(u.h),
             sum_table['sqrt_ts'],
             1,
             marker='o',
             ls='none'
            )

plt.xlabel('Livetime [h]')
plt.ylabel('Significance')

#plt.savefig('SigVsT.svg')

### Excess and Flux

We can subtract the off-source counts from the on-source counts to get the excess:

In [ ]:
spectrum_stacked.plot_counts()

#plt.savefig('Spectrum_OnOffCounts.svg')

As usual, we can use (on - alpha*off) to calculate the excess:

In [ ]:
excess = spectrum_stacked.counts.data - spectrum_stacked.counts_off.data * spectrum_stacked.alpha.data

In [ ]:
excess = excess.reshape(excess.shape[0])

In [ ]:
spectrum_stacked.geoms['geom'].axes['energy'].center

In [ ]:
plt.plot(spectrum_stacked.geoms['geom'].axes['energy'].center,
         excess,
         marker = 'o',
         ls = 'none'
        )

plt.xscale('log')
plt.yscale('log')

In [ ]:
spectrum_stacked.plot_excess()

#plt.savefig('Spectrum_ExcessCounts.svg')

We could divide the excess by the exposure to get a flux:

In [ ]:
spectrum_stacked.exposure.plot()

#plt.savefig('Spectrum_Exposure.svg')

But the binning is different and the exposure is in true energy. So we need to apply the energy dispersion matrix and to interpolate. We don't need to do that by hand. We will see later how to get the energy flux...

### Spectrum Fit
Now we want to describe the energy distribution of the gamma rays with a function. We will make a spectral fit. We have stacked all the data of all runs into one data set and proceed with the fit. We could also fit the model to each run individually.

#### Simple Power Law

We start with a simple power law. Remember, the power law is
$$
f(E) = A \times \left( \frac{E}{E_0} \right) ^{-\Gamma}.
$$
The amplitude $A$ and the spectral index $\Gamma$ are free parameters in the fit. $E_0$ is the reference energy, which is not fitted. You can freely chose the value of $E_0$, but it is best to keep it within the energy range of the data.

In [ ]:
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel

In [ ]:
spectral_model = PowerLawSpectralModel(index=2, 
                                       amplitude=1e-11 * u.Unit("cm-2 s-1 TeV-1"), 
                                       reference=1 * u.TeV
                                      )

We set the spectral model to a power law with some meaningful start parameters. The spectral model is only a part of a more general model, the SkyModel, which can also contain a spatial and a temporal model.

In [ ]:
model = SkyModel(spectral_model=spectral_model, name="Crab")

Next we set this model as the model of our Dataset. We can also have a look how well our start parameters describe the data already:

In [ ]:
spectrum_stacked.models = model

In [ ]:
spectrum_stacked.plot_excess()

plt.ylim(1e-1)

To do the fit we use a gammapy object called Fit. We can reuse the same object later, we need to create it only once.

In [ ]:
from gammapy.modeling import Fit

In [ ]:
fit = Fit()

Now we run the fit of our Dataset.

In [ ]:
fit.run(spectrum_stacked)

It is very important to check the output. Success must be True and everything has to terminate succesfully. Otherwise our fit result is wrong. If the fit fails, try to run it again. Or change the start parameters and run again. Do not continue with a failed fit!

We can now take a look at our result and the fit parameters:

In [ ]:
ax_spectrum, ax_residuals = spectrum_stacked.plot_fit()

#ax_spectrum.set_ylim(0.1, 40)

#plt.savefig('PLfit.svg')

In [ ]:
spectrum_stacked.models.to_parameters_table()

We will make a copy of the model for later use:

In [ ]:
bestmodel_PL = spectrum_stacked.models.copy()

In [ ]:
from gammapy.modeling.models import create_crab_spectral_model

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.1, 30] * u.TeV,
    "ax": ax,
}

bestmodel_PL[0].spectral_model.plot(**plot_kwargs, label="this work")
bestmodel_PL[0].spectral_model.plot_error(facecolor="blue", alpha=0.3, **plot_kwargs)

create_crab_spectral_model("hess_pl").plot(
    **plot_kwargs,
    label="Crab reference",
)

ax.legend()

#plt.savefig('PLmodel.svg')

We will also compare this model with a different model. In order to decide which model to be used we will save the test statistics.

In [ ]:
TS_PL = spectrum_stacked.stat_sum()

In [ ]:
TS_PL

#### Power Law with Exponential Cut-Off

Let's do the spectrum again, this time we want to fit a power law with an exponential cut-off at high energies. This function is defined as
$$
f(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} \times \exp \left(-\frac{E}{E_c} \right).
$$
The last term can be written as
$$
\exp \left(-\frac{E}{E_c} \right) = \exp \left(-\lambda E \right)
$$
with $\lambda = 1/E_c$.

Here it is a good idea to use our best fit parameters from the power law as starting parameters.

In [ ]:
from gammapy.modeling.models import ExpCutoffPowerLawSpectralModel

In [ ]:
bestmodel_PL.parameters['index'].quantity

In [ ]:
bestmodel_PL.parameters['amplitude'].quantity

In [ ]:
spectral_model = ExpCutoffPowerLawSpectralModel(
    index=bestmodel_PL.parameters['index'].quantity,
    amplitude=bestmodel_PL.parameters['amplitude'].quantity,
    reference=1 * u.TeV,
    lambda_ = 1./(10*u.TeV)
    )

model = SkyModel(spectral_model=spectral_model, name="Crab")

spectrum_stacked.models = model

In [ ]:
spectrum_stacked.plot_excess()

plt.ylim(1e-1)

In [ ]:
fit.run(spectrum_stacked)

Don't forget to check for success!

In [ ]:
ax_spectrum, ax_residuals = spectrum_stacked.plot_fit()

ax_spectrum.set_ylim(0.1)

#plt.savefig('ExpPLfit.svg')

In [ ]:
spectrum_stacked.models.to_parameters_table()

In [ ]:
bestmodel_expPL = spectrum_stacked.models.copy()

What is the best-fit cut-off energy?

In [ ]:
1/(bestmodel_expPL.parameters['lambda_'].quantity)

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.1, 30] * u.TeV,
    "ax": ax,
}

bestmodel_expPL[0].spectral_model.plot(**plot_kwargs, label="this work")
bestmodel_expPL[0].spectral_model.plot_error(facecolor="blue", alpha=0.3, **plot_kwargs)

create_crab_spectral_model("hess_ecpl").plot(
    **plot_kwargs,
    label="Crab reference",
)

ax.legend()

#plt.savefig('ExpPLmodel.svg')

In [ ]:
TS_expPL = spectrum_stacked.stat_sum()

In [ ]:
TS_expPL

### Compare the Models

Which model is better, with or without the cut-off? We will need to compare the test statistics of the fits.

In [ ]:
print(TS_PL, TS_expPL)

Gammapy uses a log-likelihood fit. So we can use the likelihood and Wilk's theorem, provided that we are dealing with nested models. The TS value returned by gammapy is already 2xln(L). So only need to take the difference:

In [ ]:
TS = TS_PL-TS_expPL
print(TS)

In [ ]:
import scipy.stats

In [ ]:
P = scipy.stats.chi2.sf(TS,1)

print('probabilty: ',P)

In [ ]:
print('significant?', P < 2.7e-3)

The model with the cut-off is not significantly better. So we will use the straight power law for our further analysis.

In [ ]:
bestmodel = bestmodel_PL

Let's set this model as the current model of the Dataset:

In [ ]:
spectrum_stacked.models = bestmodel

### Your playground
You can try to fit yet another model. A LogParabolaSpectralModel could work as well. Check the documentation (https://docs.gammapy.org/0.18.2/api/gammapy.modeling.models.LogParabolaSpectralModel.html#gammapy.modeling.models.LogParabolaSpectralModel) for the parameters of this model.
You can compare the log-parabola model with the power law. But you cannot use Wilk's theorem to compare it to the exponential cut-off power law, as they are not nested models.

In [ ]:
## your code here

## Flux Points
In the final step we want to create flux points which can be used for later analysis and astrophysical modelling. These points will depend on our best-fit model. Let's check first that we indeed have our best-fit model.

In [ ]:
spectrum_stacked.models[0].spectral_model.tag[0]

In [ ]:
spectrum_stacked.models.parameters.to_table()

We need only flux points within our energy range:

In [ ]:
spectrum_stacked.energy_range_total

We will use 10 bins.

In [ ]:
energy_edges = np.geomspace(0.473, 50, 11) * u.TeV

The FluxPointsEstimator will do the analysis.

In [ ]:
from gammapy.estimators import FluxPointsEstimator

In [ ]:
fpe = FluxPointsEstimator(energy_edges = energy_edges,
                          selection_optional = ['ul']
                         )

In [ ]:
flux_points = fpe.run(spectrum_stacked)

In [ ]:
flux_points.to_table()

In [ ]:
flux_points.plot()

#plt.savefig('Flux_dNdE.svg')

Flux points are often plotted in $E^2 \times dN/dE$:

In [ ]:
flux_points.plot(sed_type = 'e2dnde')

#plt.savefig('Flux_E2dNdE.svg')

If we want to plot the spectral points with our best-fit model then we best use a FluxPointsDataset.

In [ ]:
from gammapy.datasets import FluxPointsDataset

In [ ]:
flux_points_dataset = FluxPointsDataset(data = flux_points,
                                        models = bestmodel
                                       )

In [ ]:
flux_points_dataset.plot_fit()

#plt.savefig('FluxPoints_wModel.svg')

## Your playground
You can make a spectrum with more or less data points.

In [ ]:
## your code here

## Save for later use

We have created a stacked spectrum data set and a best-fit model. We can save them for later use.

In [ ]:
spectrum_stacked.write('SpectrumDataset.fits.gz',
                       overwrite = True)

In [ ]:
bestmodel.write('SpectrumBestFit.fits.gz',
                overwrite = True)

We can also save the flux points. They can be fit with an emission model.

In [ ]:
flux_points.write('SpectrumPoints.fit.gz')

So far we have used only the stacked dataset. But we also have datasets for the individual observation runs:

In [ ]:
spectrum_datasets

We can use these data sets to test for variability of the emission. We will see this at a later stage. Let's save these datasets:

In [ ]:
spectrum_datasets.write('SpectrumDatasets.yaml',
                        overwrite = True)

## Summary

We have obtained an energy spectrum of the emission and tested if a deviation from a simple power law is statistically significant. 

Our analysis depends a bit on the proper choice of the on-source region, which we have done "by eye". This can be improved.

We can also fit our data points with emission models or test for time variability of the emission.